In [8]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [9]:
rate = pd.read_csv('/Users/code/Documents/GitHub/Capstone_MacroX/Missing Value/cluster_na_rate.csv')
rate.sort_values(by='label',inplace=True)

In [51]:
rate = rate.drop_duplicates(subset='file_name')
rate

,Unnamed: 0,file_name,na_rate,label
187,187,"Uttar Pradesh_Bareilly_Civil Lines, Bareilly -...",0.994248,0
86,86,"Tamil Nadu_Chennai_Manali Village, Chennai - T...",0.515459,0
31,31,"Karnataka_Bengaluru_Hebbal, Bengaluru - KSPCB.csv",0.155127,0
30,30,"Karnataka_Bengaluru_Hombegowda Nagar, Bengalur...",0.149234,0
91,91,"Punjab_Amritsar_Golden Temple, Amritsar - PPCB...",0.527893,0
...,...,...,...,...
39,39,"Delhi_Delhi_CRRI Mathura Road, Delhi - IMD.csv",0.256486,18
119,119,"Delhi_Delhi_Burari Crossing, Delhi - IMD.csv",0.653154,18
62,62,"Delhi_Delhi_North Campus, DU, Delhi - IMD.csv",0.302771,18
14,14,"Delhi_Delhi_Patparganj, Delhi - DPCC.csv",0.029481,18


In [52]:
path = '/Users/code/Documents/GitHub/Capstone_MacroX/Processed_Gases_Official_Data/'

In [53]:
gases = ['PM2.5','SO2','NO2','CO']

In [54]:
for gas in gases:
    print(gas)
    for label in rate.label.unique():
        cur_cluster = rate[(rate['label']==label)]
        cur_min = cur_cluster['na_rate'].min()
        if cur_min < 0.06:
            standard_file = cur_cluster[(cur_cluster['na_rate']==cur_min)].reset_index()['file_name'][0]
            standard_df = pd.read_csv(path + standard_file)
            try:
                standard_df.loc[standard_df[gas] > 999, gas] = np.nan
            except:
                pass
            #Define a subset of the dataset
            df_knn = standard_df.filter([gas], axis=1).copy()
            # Define scaler to set values between 0 and 1
            df_knn = pd.DataFrame(df_knn, columns = df_knn.columns)
            # Define KNN imputer and fill missing values
            knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
            df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(df_knn), columns=df_knn.columns)
            standard_df['imputed_PM'] = df_knn_imputed[gas]
            best = standard_df[['From_Date','imputed_PM']]
            best.columns = ['From_Date',gas]

            cur_files = cur_cluster['file_name']
            start = 0

            for station in cur_files:
                if start == 0:
                    df = pd.read_csv(path+station)[['From_Date',gas]]
                    try:
                        df.loc[df[gas] > 999, gas] = np.nan
                    except:
                        pass
                    start+=1
                else:
                    if station == standard_file:
                        df = pd.concat([df,best])
                    else:
                        cur_df = pd.read_csv(path+station)[['From_Date',gas]]
                        try:
                            df.loc[df[gas] > 999, gas] = np.nan
                        except:
                            pass
                        df = pd.concat([df,cur_df])
            average = df.groupby('From_Date').mean().reset_index()

            for station in cur_files:
                if gas =='PM2.5':
                    cur_fill = pd.read_csv(path+station)
                    
                else:
                    cur_fill = pd.read_csv('/Users/code/Documents/GitHub/Capstone_MacroX/Missing Value/fill_res/'+station.split('.')[0]+'_fill.csv')
                
                cur_fill = cur_fill.merge(average, on = 'From_Date')
                res =  []
                for index,row in cur_fill.iterrows():
                    if cur_fill.isna()[gas+'_x'][index]:
                        res.append(row[gas+'_y'])
                    else:
                        res.append(row[gas+'_x'])
                cur_fill[gas+'_fill_value'] = res
                cur_fill.to_csv('/Users/code/Documents/GitHub/Capstone_MacroX/Missing Value/fill_res/'+station.split('.')[0]+'_fill.csv',index = False)

        else:
            continue
    

PM2.5
SO2
NO2
CO


In [56]:
from os import listdir
from os.path import isfile, join
mypath = '/Users/code/Documents/GitHub/Capstone_MacroX/Missing Value/fill_res/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [59]:
for file in onlyfiles:
    cur_df = pd.read_csv(mypath+file)
    cur_df = cur_df[['From_Date','To_Date','PM2.5_fill_value','NO2_fill_value','SO2_fill_value','CO_fill_value']]
    cur_df.columns = ['From_Date','To_Date','PM2.5','NO2','SO2','CO']
    cur_df.to_csv(mypath+file,index = False)

          From_Date           To_Date       PM2.5        NO2        SO2  \
0  01-01-2018 00:00  02-01-2018 00:00   49.712886  14.529958   8.006647   
1  02-01-2018 00:00  03-01-2018 00:00   63.147886  15.824958   9.091647   
2  03-01-2018 00:00  04-01-2018 00:00   62.957886  18.514958   9.696647   
3  04-01-2018 00:00  05-01-2018 00:00  129.847886  19.684958  15.131647   
4  05-01-2018 00:00  06-01-2018 00:00   69.267886  18.839958  20.951647   

         CO  
0  0.826773  
1  0.826773  
2  0.826773  
3  0.826773  
4  0.826773  
          From_Date           To_Date   PM2.5    NO2        SO2    CO
0  01-01-2018 00:00  02-01-2018 00:00  282.61  33.62  26.793333  3.70
1  02-01-2018 00:00  03-01-2018 00:00  274.29  31.08  15.433333  1.92
2  03-01-2018 00:00  04-01-2018 00:00  317.91  28.85  15.770000  1.53
3  04-01-2018 00:00  05-01-2018 00:00  360.42  27.31  15.006667  2.47
4  05-01-2018 00:00  06-01-2018 00:00  406.01  24.32  11.566667  1.96
          From_Date           To_Date    PM2.